In [9]:
#逻辑回归
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import roc_auc_score
lr = LogisticRegression(class_weight='balanced')
# param_dict = {'C':[0.5,1.0,2.0],'tol':[0.0001,0.00001,0.000001]}
# lr_gdc = GridSearchCV(lr,param_grid=param_dict,cv=3)
# lr_gdc.fit(X_train,y_train)
lr.fit(X_train,y_train)
# y_predict_proba = lr_gdc.predict_proba(X_test)
y_predict_proba = lr.predict_proba(X_test)[:, 1]
lr_auc = roc_auc_score(y_test,y_predict_proba)
print(lr_auc)
# print('最佳参数：',lr_gdc.best_params_)   

0.7304725755227364


In [16]:
#blend融合
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.datasets.samples_generator import make_blobs
import matplotlib.pyplot as plt

def data_show(X,y):
    plt.scatter(X[:,0],X[:,1],c=y)
    plt.show()


def blend(X,X_predict,y,y_predict):
    '''5折'''
    # n_flods = 5
    # skf = list(StratifiedKFold(y, n_folds=n_flods))
    '''切分训练数据集为d1,d2两部分'''
    X_d1, X_d2, y_d1, y_d2 = train_test_split(X, y, test_size=0.7, random_state=2017)

    dataset_d1 = np.zeros((X_d2.shape[0],len(clfs)))
    dataset_d2 = np.zeros((X_predict.shape[0],len(clfs)))

    for j,clf in enumerate(clfs):
        '''依次训练各个单模型'''
        # print(j, clf)
        '''使用第1个部分作为预测，第2部分来训练模型，获得其预测的输出作为第2部分的新特征。'''
        # X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]
        clf.fit(X_d1,y_d1)
        y_submission = clf.predict_proba(X_d2)[:, 1]
        dataset_d1[:,j] = y_submission
        '''对于测试集，直接用这k个模型的预测值作为新的特征。'''
        dataset_d2[:, j] = clf.predict_proba(X_predict)[:, 1]
        print("val auc Score: %f" % roc_auc_score(y_predict, dataset_d2[:, j]))

    '''使用模型融合'''
    clf = GradientBoostingClassifier(learning_rate=0.2,subsample=0.5,max_depth=6,n_estimators=30)
    clf.fit(dataset_d1,y_d2)
    y_submission = clf.predict_proba(dataset_d2)[:,1]
    print("Linear stretch of predictions to [0,1]")
    y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())
    print("blend result")
    print("val auc Score: %f" % (roc_auc_score(y_predict, y_submission)))


if __name__ == "__main__":
    data =  pd.read_csv('treated_data.csv')
    data,target = data.drop(['target'],axis=1),data['target']
    
    # data_show(data,target)#查看数据分布
    '''模型融合中使用到的各个单模型'''
    clfs = [LogisticRegression(class_weight='balanced'),
            RandomForestClassifier(n_estimators=400, min_samples_leaf=3,max_depth=12,max_features=0.14357,min_samples_split=6,n_jobs=-1, criterion='gini'),        
            RandomForestClassifier(n_estimators=400, min_samples_leaf=7,max_depth=6,max_features=0.07,min_samples_split=6,n_jobs=-1, criterion='entropy'),
            ExtraTreesClassifier(n_estimators=400, n_jobs=-1, criterion='gini'),
            ExtraTreesClassifier(n_estimators=400, n_jobs=-1, criterion='entropy')]
#             GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=5)
#             ]

    data =  pd.read_csv('treated_data.csv')
    data,target = data.drop(['target'],axis=1),data['target']
    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.33,random_state=2019)
    transformer = StandardScaler()
    X_train = transformer.fit_transform(X_train)
    X_test = transformer.transform(X_test)
    blend(X_train,X_test,y_train,y_test)

val auc Score: 0.683728
val auc Score: 0.677184
val auc Score: 0.701283
val auc Score: 0.671532
val auc Score: 0.694978
Linear stretch of predictions to [0,1]
blend result
val auc Score: 0.692051


In [18]:
#软投票融合
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
clf1=LogisticRegression(class_weight='balanced')
clf2=RandomForestClassifier(n_estimators=400, min_samples_leaf=3,max_depth=12,max_features=0.14357,min_samples_split=6,n_jobs=-1, criterion='gini')         
clf3=ExtraTreesClassifier(n_estimators=400, n_jobs=-1, criterion='entropy')
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('et', clf3)],voting='soft')
eclf = eclf.fit(X_train,y_train)
y_predict_proba = eclf.predict_proba(X_test)[:, 1]
lr_auc = roc_auc_score(y_test,y_predict_proba)
print(lr_auc)



0.7481624447988132


In [19]:
from sklearn.externals import joblib
joblib.dump(eclf, "final_model.pkl")

['final_model.pkl']